In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import os

In [5]:
import itertools

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models

In [8]:
data = pd.read_csv("bbc-text.csv")

In [9]:
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [10]:
data.tail()

,category,text
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...
2224,sport,souness delight at euro progress boss graeme s...


In [11]:
data['category'].value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [12]:
train_size = int(len(data) * .9)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 2002
Test size: 223


In [13]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [14]:
train_cat, test_cat = train_test_split(data['category'], train_size)
train_text, test_text = train_test_split(data['text'], train_size)

In [16]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=False)

In [17]:
tokenize.fit_on_texts(train_text)
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [18]:
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [19]:
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [20]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (2002, 1000)
x_test shape: (223, 1000)
y_train shape: (2002, 5)
y_test shape: (223, 5)


In [21]:
batch_size = 64
epochs = 5
drop_ratio = 0.5

In [22]:
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)

Epoch 1/5


29/29 [==============================] - 1s 13ms/step - loss: 0.5402 - accuracy: 0.8418 - val_loss: 0.2021 - val_accuracy: 0.9403
Epoch 2/5
29/29 [==============================] - 0s 6ms/step - loss: 0.0737 - accuracy: 0.9833 - val_loss: 0.1374 - val_accuracy: 0.9602
Epoch 3/5
29/29 [==============================] - 0s 5ms/step - loss: 0.0263 - accuracy: 0.9978 - val_loss: 0.1139 - val_accuracy: 0.9602
Epoch 4/5
29/29 [==============================] - 0s 6ms/step - loss: 0.0112 - accuracy: 1.0000 - val_loss: 0.1038 - val_accuracy: 0.9652
Epoch 5/5
29/29 [==============================] - 0s 7ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.1006 - val_accuracy: 0.9652


In [24]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

4/4 [==============================] - 0s 6ms/step - loss: 0.1465 - accuracy: 0.9552
Test loss: 0.14648978412151337
Test accuracy: 0.9551569223403931


In [25]:
def run_experiment(batch_size, epochs, drop_ratio):
  print('batch size: {}, epochs: {}, drop_ratio: {}'.format(
      batch_size, epochs, drop_ratio))
  model = models.Sequential()
  model.add(layers.Dense(512, input_shape=(max_words,)))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(drop_ratio))
  model.add(layers.Dense(num_classes))
  model.add(layers.Activation('softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
  score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=0)
  print('\tTest loss:', score[0])
  print('\tTest accuracy:', score[1])

In [26]:
batch_size = 16
epochs = 4
drop_ratio = 0.4
run_experiment(batch_size, epochs, drop_ratio)

batch size: 16, epochs: 4, drop_ratio: 0.4
	Test loss: 0.18520991504192352
	Test accuracy: 0.9417040348052979


In [27]:
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_text.iloc[i][:50], "...")
    print('Actual label:' + test_cat.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

1/1 [==============================] - 0s 79ms/step
salary scandal in cameroon cameroon says widesprea ...
Actual label:business
Predicted label: politics

1/1 [==============================] - 0s 21ms/step
mild winter drives us oil down 6% us oil prices ha ...
Actual label:business
Predicted label: business

1/1 [==============================] - 0s 18ms/step
johansson takes adelaide victory second seed joach ...
Actual label:sport
Predicted label: sport

1/1 [==============================] - 0s 25ms/step
police urge pub closure power new powers are neede ...
Actual label:politics
Predicted label: politics

1/1 [==============================] - 0s 20ms/step
mirza shocks kuznetsova sania mirza continued her  ...
Actual label:sport
Predicted label: sport

1/1 [==============================] - 0s 38ms/step
new delay hits eu software laws a fresh delay has  ...
Actual label:tech
Predicted label: tech

1/1 [==============================] - 0s 43ms/step
half of uk s mobiles  go online 